# Short-Term Investment Strategies with Deep Q-Network (DQN)

## Introduction

The goal of this Jupyter Notebook is to explore the application of **Deep Reinforcement Learning**, and in particular the **Deep Q-Network (DQN)** algorithm, in the domain of financial investments, with a focus on **short-term investment strategies**.

The investment landscape can be broadly divided into two main categories.  
1. **Long-term investments**, typically rely on macroeconomic factors, political events, corporate decisions, and geopolitical dynamics, and are often analyzed through [fundamental analysis](https://en.wikipedia.org/wiki/Fundamental_analysis).  
2. **short-term investments**, which include trading activities, operate over much shorter time horizons and are commonly studied using mathematical and statistical models.

Traditional approaches in quantitative finance for short-term price modeling include tools such as **ARIMA models**, **Geometric Brownian Motion (GBM)**, and model-based frameworks like the **Black–Scholes model**, (if you are interested, see my [arXiv paper here](https://arxiv.org/pdf/2510.27277)). While powerful, these methods rely on strong assumptions about market dynamics.

In this notebook, we adopt a different perspective by leveraging **Artificial Intelligence**, in particular **Reinforcement Learning (RL)**.\
Instead of explicitly modeling price dynamics, we allow an agent to **learn a trading policy directly from interactions with the market environment**, guided by a reward signal.

A crucial step in this process is the design of a suitable **trading environment**. The environment must comply with the standard **Gymnasium** interface and provide:
- a well-defined **action space** (e.g.: buy, sell, hold),
- an **observation space** representing the market state,
- proper **termination and truncation conditions**,
- and, most importantly, a **reward function** that encodes the trading objective.

Once the environment is defined, it is coupled with a **DQN agent**, which is trained through repeated interactions with the market. Finally, the trained agent's policy is evaluated by analyzing the accumulated rewards compared to a random policy, in order to assess the effectiveness and robustness of the learned trading strategy.


<div style="text-align: center;">
    <img src="trading_bot.JPG" alt="trading bot image" width="66%"/>
</div>


## Current SOTA

It is important to point out that [Gymnasium](https://gymnasium.farama.org/), which is a standardized API for RL ready-to-use environments, currently does not have any predefined trading environment, all we have are some projects carried out 'unofficially', that often lack of correctness and rigorous methoods or documentation.

* [gym-anytrading](https://github.com/AminHP/gym-anytrading), repository GitHub for a possible trading environment.
* [tensortrade](https://github.com/tensortrade-org/tensortrade), Python library for reinforcement learning applied in trading.
* [q-trader](https://github.com/edwardhdlu/q-trader?tab=readme-ov-file), application of reinforcement learning in the stock market.

These three, seem to be the most relevant sources we can find on the web, but as already noted, they lack of proper documentation and rigorous method.

In any case, the application of RL techniques in investments is already quite widespread as mentioned by this article [link](https://medium.com/ibm-data-ai/reinforcement-learning-the-business-use-case-part-2-c175740999), where prestigious firms suchs IBM or J.P. Morgan, already apply some of these techniques.

## The *environment* and  *reward* signal

Given the current SOTA available and having understood the main dynamics of a market stock, the first thing to do, is the design of our own environment.\
The Python script `tradingenv` contains our environment class, `TradingEnv()`. It could be used as a starting point for many other RL projects, since it contains a proper methodology with many comments that enhance the understanding of the development.

TradingEnv has all the methods that make it compliant to the Gym environments. But the most relevant thing to understand is how the **reward** signal is defined, as shown here:

$ reward = (cash_{t+1} + shares_{t+1} \cdot p_{t+1}) - (cash_{t} + shares_{t} \cdot p_{t}) $

where:
* $reward$, indicates the reward obtained after taking the action at time-step $t$
* $cash_{t}$, indicates the amount of cash held at time step $t$
* $share_{t}$, indicates the amount of shares held at time step $t$
* $p_{t}$, indicates the price of a single share at time step $t$

## Development and Implementation

import of all the necessary libraries, environment, numerical computations, NN and rendering...\
Make sure to have the script `tradingenv`!

In [ ]:
#import of our defined environment
#YOU MUST ALSO HAVE THE SCRIPT "tradingenv.py"
from tradingenv import TradingEnv

import torch #for Neural Networks
import numpy as np #for numerical operations
import random #for random number generation
import joblib #for saving and loading the replay memory
import matplotlib.pyplot as plt #for plotting the results

Example of how to use the library

In [ ]:
""" example of how to use the described environment
env = TradingEnv(ticker="AAPL", granularity="1d", sliding_window=7, start_date="2020-01-01", end_date="2021-01-01", initial_cash=10000, initial_share=5)
obs, info = env.reset()
action = env.action_space.sample()  # es. 0=buy,1=hold,2=sell
obs, reward, done, truncated, info = env.step(action)
"""

### Instantiation of the trading environment with the selected stock ticker of interest

Note, you can select the granularity as the interval parameter of the `yfinance` [library](https://ranaroussi.github.io/yfinance/), representing the time scale over which you want to operate.\
Supported time scale: 

#### Short-time trading (intraday)

* `1m` – 1 minute
* `2m` – 2 minutes
* `5m` – 5 minutes
* `15m` – 15 minutes
* `30m` – 30 minutes
* `60m` – 60 minutes
* `90m` – 90 minutes
* `1h` – 1 hour

#### Long-time trading (daily and others)
* `1d` – 1 day
* `5d` – 5 days
* `1wk` – 1 week
* `1mo` – 1 month
* `3mo` – 3 months

In [ ]:
#instatiate the environment
env = TradingEnv(ticker="AAPL", granularity="1d", sliding_window=10, start_date="2022-01-01", end_date="2025-01-01", initial_cash=100000, initial_share=50)

state_size = env.observation_space.shape
action_size = env.action_space.n

print('State Space: ', state_size)
print('Action Space: ', action_size)

### Note that here we have some fundamental hypeparamenters to tweak

* The `start_date` and `end_date` will generate the amount of data used for training in the period of interest, we must tweak them according to our purpose, but generally speaking they should cover a period of circa 5 years, in order to enclose possible financial market cycles.
* The `slidinig_window` should be approximately 10 days with a maximum of 20, but for the sake of computational efficiency we have decided to set it to 7 days (keep in mind that it also depends on the type of trading the user want to perform).
* The `granularity` hyperparameter as already mentioned is crucial, and depends on the type of trading the user wants to perform.
* The `initial_cash` and `initial_share` combined, represent the agent initial wallet value, and are of crucial importance for the learning of behaviour, **we must give to the agent a proper amount of stock and cash available at the beginning, otherwise it will learn degenerative policy.**\
**Note that the two amounts should depend on the market considered.**

### Observation Space: Relative Price Variations

It is important to emphasize that the observation space is defined in terms of **percentage price variations** rather than **absolute price levels**.  
This choice is fundamental from both a financial and a reinforcement learning perspective.

Absolute prices are not directly comparable across assets or time periods: a price change of +1 has a very different meaning for an asset trading at 10 compared to one trading at 1,000. By using relative variations, the agent receives **scale-invariant information**, allowing it to learn patterns that are independent of nominal price levels.

Moreover, financial markets are highly **non-stationary** in absolute terms, as prices tend to drift over time. Percentage variations, instead, exhibit more stable statistical properties, making them more suitable as inputs for learning algorithms and improving training stability.

Finally, relative variations naturally align with the **economic objective of trading**, since profits and losses are inherently relative quantities. For these reasons, the observation space is constructed using relative price changes, enabling better generalization and a more meaningful learning process for the DQN agent.

If you are interested, here is a similar project of mine where I needed stationarity in time series [Fast Fourier Transform in Finance](https://github.com/fraro01/Fourier-Transform-in-Finance)

## What our agent is training on

Show what we are analyzing, the time series of interest, note that we use as prices the closing prices given by `yfinance`. This behaviour can be changed by simpply tweaking the related hyperparameter from the `tradingenv.py`

In [ ]:
#method of the TradingEnv class to show the data used for training
env.show_data()

We fix the seed of the random number generator in order to reproduce the results

In [ ]:
# set the seeds for reproducibility of results
seed = 34

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

env.reset(seed=seed)
env.action_space.seed(seed)
env.observation_space.seed(seed)

If available on the current calculator, we take advantage of the **GPU** computational power

In [ ]:
# set the backend device to MPS or CUDA, if available
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# print the used device
print(f"Using device: {device}")

## Experience Replay Buffer

Experience Replay Buffer, for the creation of semi-independent experiences to train the agent, in order to adress the first issue given by: *i.i.d*

Definition of the single array of experience

In [ ]:
# define the structured dtype for an experience tuple, which will be stored in the replay memory
experience_type = np.dtype([
    ('state',      np.float32, state_size),   # current state, it must take the warray of the observation space
    ('action',     np.int8),                  # action taken, it is JUST THE action we took
    ('reward',     np.float32),               # reward received
    ('next_state', np.float32, state_size),   # next state
    ('failure',    np.int8)                   # terminal flag (1 if done)
])

Definition of the container of the arrays (replay buffer)

In [ ]:
# Set the replay memory size hyperparameter
memory_size = 100000 #TODO! ADJUST IT ACCORDINGLY, it waas set to 100000 at the beginning

# Create the replay memory
replay_memory = {
    'size': memory_size,
    'buffer': np.empty(shape=(memory_size,), dtype=experience_type),
    'index': 0,
    'entries': 0
}

Function to store a new experience in the replay buffer

In [ ]:
def store_experience(experience):
    # store the experience in the buffer
    replay_memory['buffer'][replay_memory['index']] = experience #we overwrite the experience at the current index, if the buffer is not full it will be empty, otherwise it will overwrite the oldest experience

    # update the number of experiences in the buffer
    replay_memory['entries'] = min(replay_memory['entries'] + 1, replay_memory['size']) # it cannot exceed the maximum size of the buffer

    # update index, if the memory is full, start from the begging
    replay_memory['index'] += 1
    replay_memory['index'] = replay_memory['index'] % replay_memory['size'] #as long as 'index' is lower than 'size' it outputs 'index', then once they are equal it outputs 0

Function to sample a mini-batch of experiences from the replay buffer defined above.\
We will train the agent over these mini-batches.

In [ ]:
# Set the batch size for sampling experiences
batch_size = 32 #hyperparameter to be tweaked TODO! it was 32

# function to sample a batch of experiences from the replay memory
def sample_experiences():

    # select uniformly at random a batch of experiences from the memory
    idxs = np.random.choice(range(replay_memory['entries']), batch_size, replace=False) #without replacement, we want unique experiences in the batch, if 'entries' is lower than 'batch_size' it will raise an error, but in that case we should not sample a batch of experiences, we should wait until we have enough experiences in the buffer

    # return the batch of experiences
    experiences = replay_memory['buffer'][idxs]

    return experiences

## Neural Network Architecture

It will output the $Q(s,a)$ for all possible actions in the given state $s$, after the observation taken as input.\
Here many hyperparameters could be tweaked, such as the hidden layers.

In [ ]:
#hyperparameters for the neural network architecture to be tweaked
first_hidden_layer= 512
second_hidden_layer= 128

def create_network():

      # Define a deep neural network using Sequential:
      # Each layer feeds directly into the next one.
      dnn = torch.nn.Sequential( 
            # First fully connected layer maps state inputs to 512 hidden units
            torch.nn.Linear(state_size[0], first_hidden_layer),
            
            # ReLU activation introduces nonlinearity
            torch.nn.ReLU(),
            
            # Second fully connected layer (hidden layer with 128 units)
            torch.nn.Linear(first_hidden_layer, second_hidden_layer),
            
            # Another ReLU activation
            torch.nn.ReLU(),
            
            # Output layer: one unit per possible action
            # Produces Q-values
            torch.nn.Linear(second_hidden_layer, action_size)
      )
    
      # Return the constructed model
      return dnn

Creation of two separated networks (in terms of weights), in order to adress the second issue in *i.i.d.*, of *identically distributed* targets.

$\displaystyle L(\theta) = E_{(s,a) \sim U(D)} \left[ ( r + \gamma \underset{a}{\text{ max }} Q(s',a;\theta^{-}) - Q(s,a;\theta))^2 \right]$

In [ ]:
online_q = create_network() # online q-network for the prediction
target_q = create_network() # target network for the Ground-Truth Q-value estimation

Optimizer only for the online network, since is the only one to be trained

In [ ]:
learning_rate = 0.007
optimizer = torch.optim.RMSprop(online_q.parameters(), lr=learning_rate)

The target network is not trained, but sometimes we need to **update its weights to match the online network weights**

In [ ]:
def update_target():
    # copy the parameters from the online model to the target model
    for target, online in zip(target_q.parameters(), online_q.parameters()):
        target.data.copy_(online.data)

Optimizer, for computing the loss and the backward propagation

In [ ]:
# define the discount factor
gamma = 0.99

def optimize():

    # sample a batch of experiences
    batch = sample_experiences()
    
    # prepare the experience as tensors
    states      = torch.from_numpy(batch['state'].copy()).float()    
    actions     = torch.from_numpy(batch['action'].copy()).long()   
    rewards     = torch.from_numpy(batch['reward'].copy()).float()    
    next_states = torch.from_numpy(batch['next_state'].copy()).float() 
    failures    = torch.from_numpy(batch['failure'].copy()).float()

    # get the values of the Q-function at next state from the "target" network 
    # remember to detach, we need to treat these values as constants 
    q_target_next = target_q(next_states).detach()
    
    # get the max value 
    max_q_target_next = q_target_next.max(1)[0]

    # one important step, often overlooked, is to ensure 
    # that failure states are grounded to zero
    max_q_target_next *= (1 - failures.float())

    # calculate the target 
    target = rewards + gamma * max_q_target_next

    # finally, we get the current estimate of Q(s,a)
    # here we query the current "online" network
    q_online_current = torch.gather(online_q(states), 1, actions.unsqueeze(1)).squeeze(1)

    # create the errors
    td_error = target - q_online_current

    # calculate the loss
    loss = td_error.pow(2).mean()

    # backward pass: compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # update model parameters
    optimizer.step()

## Exploration Vs Exploitation

Hyperparameters to be set for the exploration straetegies. *($\epsilon$ decay)*

In [ ]:
# define decay parameters (max, min, steps)
epsilon_max = 1.0
epsilon_min = 0.01
epsilon_decay_steps = 10000

# generate epsilons
epsilons = np.logspace(start=0, stop=-2, num=epsilon_decay_steps, base=10)
   
# normalize epsilons 
epsilons = (epsilons - epsilon_min) / (epsilon_max - epsilon_min)
    
# scale  epsilons to the desired range
epsilons = (epsilon_max - epsilon_min) * epsilons + epsilon_min

Chosen exploration strategy.\
Based on the picked $\epsilon$, we alternate a random action selection and the greedy selection (greedy based on the value of the $Q(s,a,\theta)$ returned by our neural network).

`random_pi()` is the baseline random policy, to be used as reference, in order to understand whether we are actually learning smoething

In [ ]:
def random_pi(state):
    # selects an action uniformly at random
    # from the environment's action space.
    return env.action_space.sample()

At the end of training the online network will be used to implement the policy.\
`dqn_policy()` it simply acts greedy given the input state.

In [ ]:
def dqn_pi(state):
    # convert the state into a tensor
    state = torch.as_tensor(state, dtype=torch.float32)

    # compute Q-values from the network
    q_values = online_q(state).detach().numpy().squeeze()

    # select greedy action
    action = int(np.argmax(q_values))

    # return the action
    return action

`epsilon_greedy()` combines `random_pi()` and `dqn_pi()` in order to be indeed $\epsilon\$-greedy decaying

In [ ]:
def epsilon_greedy(state, step):
    # get the epsilon value    
    epsilon = epsilons[step] if step < epsilon_decay_steps else epsilon_min

    # Exploration
    if random.random() < epsilon:
        action = random_pi(state)

    # Exploitation
    else:
        action = dqn_pi(state)
    
    return action

## Policy Evaluation

Function to evaluate a policy, it returns the average reward obtained over a number of episodes

In [ ]:
def evaluate(pi, episodes=1):

     # collect total rewards per episode
    rewards = []

    # loop over episodes
    for episode in range(episodes):

        # reset the environment
        state, _ = env.reset()
        done = False
        total_reward = 0.0

        # run an episode
        while not done:
            action = pi(state)
            state, reward, terminal, truncated, _ = env.step(action)
            total_reward += reward
            done = terminal or truncated

        # store the total reward    
        rewards.append(total_reward)
            
    # return the average reward over the episodes        
    return np.mean(rewards)

## Deep Q-Network Algorithm
Putting all the things together

In [ ]:
def dqn(memory_start_size, target_update_steps, max_episodes):
    
    # create a score tracker for statistic purposes
    scores = []
    
    # counter for the number of steps 
    step = 0

    # update the target model with the online one
    update_target()
                   
    # train until the maximum number of episodes
    for episode in range(max_episodes):
        
        # reset the environment before starting the episode
        state, _ = env.reset()
        done = False

        # interact with the environment until the episode is done
        while not done:
                    
            # select the action using the exploration policy
            action = epsilon_greedy(state, step)

            # perform the selected action
            next_state, reward, terminal, truncated, _ = env.step(action)
            done = terminal or truncated
            failure = terminal and not truncated

            # store the experience into the replay buffer
            experience = (state, action, reward, next_state, failure)
            store_experience(experience)
    
            # optimize the online model after the replay buffer is large enough
            if replay_memory['entries'] > memory_start_size:
                optimize()
                 
                # sometimes, synchronize the target model with the online model
                if step % target_update_steps == 0:
                    update_target()
                
            # update current state to next state
            state = next_state

            # update the step counter
            step += 1

        # After each episode, evaluate the policy
        score = evaluate(dqn_pi, episodes=10)

        # store the score in the tracker
        scores.append(score)

        # print some informative logging 
        message = 'Episode {:03}, score {:05.1f}'
        message = message.format(episode+1, score)
        print(message, end='\r', flush=True)
        
    return scores

Apply the DQN to our `TradingEnv`

In [ ]:
# set the hyperparameters
memory_start_size = 1000
max_episodes = 100 #TODO! 100
target_update_steps = 10

# run the DQN algorithm
dqn(memory_start_size, target_update_steps, max_episodes)

## Compare DQN to random_pi

Experiment to understand how dqn is learning over the episodes

In [ ]:
def experiment(max_episodes):

    global online_q, target_q, optimizer, replay_memory, epsilons

    # List of random seeds to test algorithm stability
    seeds = (12, 34, 56, 78, 90)

    # Container to collect all experiment results
    results = []

    # Run an independent training experiment per seed
    for seed in seeds:

        print("Experiment seed: ", seed)

         # Set all relevant random seeds for reproducibility
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # reset the environment
        env.reset(seed=seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)

        # create online and target models
        online_q = create_network()
        target_q = create_network()
        optimizer = torch.optim.RMSprop(online_q.parameters(), lr=learning_rate)

        # create the replay memory
        replay_memory = {
            'size': memory_size,
            'buffer': np.empty(shape=(memory_size,), dtype=experience_type),
            'index': 0,
            'entries': 0
        }

        # create the epsilon values
        epsilons = np.logspace(start=0, stop=-2, num=epsilon_decay_steps, base=10)
        epsilons = (epsilons - epsilon_min) / (epsilon_max - epsilon_min)
        epsilons = (epsilon_max - epsilon_min) * epsilons + epsilon_min

        # train the network    
        scores = dqn(memory_start_size, target_update_steps, max_episodes)
        
        # smooth the result using a sliding window
        sliding_windows = 25
        scores = np.convolve(scores, np.ones(sliding_windows)/sliding_windows, mode='valid')
                
        # collect the results
        results.append(scores)

        print("")

    # calculate max, min and average scores among experiments
    max_score = np.max(results, axis=0).T
    min_score = np.min(results, axis=0).T
    mean_score = np.mean(results, axis=0).T

    # prepare the results
    experiment_results = {
        'max_score': max_score,
        'min_score': min_score,
        'mean_score': mean_score
    }

    # save permanently
    joblib.dump(experiment_results, '../dqn_results.joblib');
    
    return experiment_results

Run of the experiment on our environemnt

In [ ]:
# Define the experiment setup and hyperparameters
gamma = 0.99               # discount factor
learning_rate = 0.001      # step size for the optimizer
batch_size = 512          # number of experiences per batch
epochs = 8                # optimization steps per batch
epsilon = 0.5               # esploration vs exploitation parameter
first_hidden_layer = 256   # size of the first hidden layer
second_hidden_layer = 128  # size of the second hidden layer

# Run the experiment
dqn_results = experiment(max_episodes=1500) #TODO! 1500

Experiment to see the performance of a random behaviour

In [ ]:
def experiment_random(max_episodes):

    # List of random seeds to test algorithm stability
    seeds = (12, 34, 56, 78, 90)

    # Container to collect all experiment results
    results = []

    # Run an independent training experiment per seed
    for seed in seeds:

        print("Experiment seed: ", seed)

         # Set all relevant random seeds for reproducibility
        np.random.seed(seed)
        random.seed(seed)

        # reset the environment
        env.reset(seed=seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)

        # train the network    
        scores = evaluate(random_pi, episodes=max_episodes)
        
        # smooth the result using a sliding window
        sliding_windows = 25
        scores = np.convolve(scores, np.ones(sliding_windows)/sliding_windows, mode='valid')
                
        # collect the results
        results.append(scores)

        print("")

    # calculate max, min and average scores among experiments
    max_score = np.max(results, axis=0).T
    min_score = np.min(results, axis=0).T
    mean_score = np.mean(results, axis=0).T

    # prepare the results
    experiment_results = {
        'max_score': max_score,
        'min_score': min_score,
        'mean_score': mean_score
    }

    
    return experiment_results

Run the experiment for the randomic behaviour

In [ ]:
random_pi_results = experiment_random(max_episodes=1500) #TODO! 1500

### Comparison between DQN and random_pi in learning in terms of robustness with confidence bands

In [ ]:
#define the plots
plt.figure(figsize=(12,6))
plt.title('DQN vs Random Policy')
plt.ylabel('Average reward per episode [$]')
plt.xlabel('Episodes')

#get the results of the 2 experiments
dqn_episodes = range(len(dqn_results['max_score']))
random_pi_episodes = range(len(random_pi_results['max_score']))

plt.plot(random_pi_results['max_score'], 'y', linewidth=1, label="random_pi")
plt.plot(random_pi_results['min_score'], 'y', linewidth=1)
plt.plot(random_pi_results['mean_score'], 'y', linewidth=2)
plt.fill_between(random_pi_episodes, random_pi_results['min_score'], random_pi_results['max_score'], facecolor='y', alpha=0.3)

plt.plot(dqn_results['max_score'], 'b', linewidth=1, label="DQN")
plt.plot(dqn_results['min_score'], 'b', linewidth=1)
plt.plot(dqn_results['mean_score'], 'b', linewidth=2)
plt.fill_between(dqn_episodes, dqn_results['min_score'], dqn_results['max_score'], facecolor='b', alpha=0.3)

plt.legend()
plt.show()

## What our agent is capable of in reality

It is crucial to clarify a fundamental aspect of reinforcement learning applied to financial trading, we are somehow *in the middle of reinforcement learning and supervised learning problem*.\
While the problem is formally framed as a reinforcement learning task, it exhibits several characteristics that make it closer to an offline and supervised-like learning setting, as in between a problem of pattern recognition.

In particular, the actions taken by the agent do not influence the environment dynamics: the price time series is pre-generated and entirely exogenous. As a consequence, the environment is non-reactive, and the agent interacts with a fixed historical trajectory rather than a fully interactive Markov Decision Process. <small>(see the final MarkDown cell discussing Markovian assumptions)</small>\
[See the fiancial market whale](https://fenefx.com/en/blog/what-is-a-financial-market-whale/)

This property introduces a significant risk of overfitting. Evaluating the cumulative reward obtained by the agent on the same time series used during training does not provide meaningful information about its true generalization capability. In such a setting, the agent may simply learn spurious correlations or regime-specific patterns present in the training data, achieving high in-sample performance without learning a robust trading strategy.

Therefore, performance evaluation must be conducted on strictly unseen time series, either from future time periods or from different assets, in order to assess the agent’s ability to generalize to a real and previously unobserved market environment. Only out-of-sample evaluation can provide a realistic estimate of the strategy’s effectiveness and risk profile.

Thus, we will evaluate the performance of our agent over a shifted and shorter environment, **over the same market**, this last thing it is fundamental again.\
We should test our policy on the same market over which it has been trained on, because different markets might have different types of characteristics, such as: market cap, amount of investors, regulatory rules, volatility...

In [ ]:
#instantiate a new environment, we will use it for testing the trained policy
#choose a shorter start_date - end_date range to speed up the evaluation phase
env = TradingEnv(ticker="AAPL", granularity="1d", sliding_window=10, start_date="2025-01-01", end_date="2026-01-01", initial_cash=100000, initial_share=50)

state_size = env.observation_space.shape
action_size = env.action_space.n

print('State Space: ', state_size)
print('Action Space: ', action_size)

In [ ]:
#method of the TradingEnv class to show the data used testing the trained policy
env.show_data()

In [ ]:
#we can even run it many times with many different input settings to draw a Design of Experiments curve
evaluate(dqn_pi, episodes=100)

In [ ]:
#just for comparison test the perfomance of the random policy
evaluate(random_pi, episodes=100)

## Conclusions, Limitations and Future Improvements

The results highlight a clear difference between a **random policy** and the policy learned by the DQN agent.  
This suggests that the agent is able to capture **non-trivial patterns** in market dynamics, potentially exploiting regularities that go beyond pure randomness. However, these results should be interpreted with caution, as financial markets are complex and highly noisy systems.

It is well known in the financial literature that markets may exhibit recurring structures, such as [CandleStick patterns](https://en.wikipedia.org/wiki/Candlestick_pattern), which reflect collective trader behavior and market psychology. Reinforcement learning methods may implicitly learn such patterns without explicitly encoding them, like in pattern recognition.

### Limitations

Despite the encouraging results, the proposed model suffers from several limitations:

- **Strong assumptions on the action space**:  
  The agent can only buy, sell, or hold a single unit of the asset at each step. This is a restrictive assumption that does not reflect real trading behavior.

- **Simplified state representation**:  
  The observation space is limited to a fixed-size sliding window of past price variations, excluding other potentially informative features such as volume, volatility indicators, or macroeconomic signals.

- **Sensitivity to hyperparameters**:  
  The performance of the model strongly depends on the choice of neural network architecture and DQN hyperparameters, which have not been exhaustively optimized.\
  Furthermore even the environment is strongly dependent on hyperparamters, that should be properly defined by a deep financial knowledge.\
  Even the initialization of the cash and share hyperparameters is crucial, because from this, the agent could learn degenerative behaviours.

- **Limited time horizon and computational constraints**:  
  Computational power limits the amount of historical data and the length of training, potentially preventing the agent from learning long-term dependencies.

### Future Improvements

Several directions can be explored to improve the model:

- **Hyperparameter optimization**:  
  Perform a systematic search (e.g. grid search or random search) over network architectures and learning parameters.

- **Different time resolutions**:  
  Extend the framework to different time scales, such as intraday or hourly trading, to analyze high-frequency dynamics.

- **Richer action space**:  
  Allow the agent to buy or sell multiple units per action, enabling position sizing and more realistic portfolio management.

- **Expanded state space**:  
  Incorporate additional features such as technical indicators, trading volume, volatility measures, or external signals.

- **Real-world deployment**:  
  After proper validation and risk management, the trained agent could be connected to a trading bot interfaced with a real (or paper) trading wallet.

- **Use of finer models**:  
  We could try to use different models, such as DDQN, Rainbow or even policy-based methods.

Overall, while the results are promising, further experimentation and validation are required before considering real-world applications.

### On the Markov Decision Process formulation in financial trading

A Markov Decision Process (MDP) is defined by a tuple $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$, where the transition function $P(s_{t+1} \mid s_t, a_t)$ explicitly depends on the action taken by the agent.\
This dependency implies that actions actively shape the future evolution of the environment.

In the context of financial trading on historical price data, this assumption does not strictly hold.\
The price time series is pre-generated and entirely exogenous, meaning that the agent’s actions do not influence the future market states.
As a consequence, the state transition dynamics satisfy:

$P(s_{t+1} \mid s_t, a_t) = P(s_{t+1} \mid s_t)$

The agent’s actions affect only the reward function, through realized profits and losses, while the environment evolution remains unchanged.\
This results in a non-reactive environment, where the agent interacts with a fixed historical trajectory rather than a fully interactive MDP.

Therefore, the trading problem is more accurately described as an offline reinforcement learning problem with exogenous dynamics, or as a degenerate MDP.\
This formulation highlights the increased risk of overfitting and the necessity of strict out-of-sample evaluation, as the agent may otherwise exploit idiosyncratic patterns specific to the training data rather than learning a robust trading policy.
